# Puesta a punto



## Instalación de paquetes necesarios







In [1]:
!pip3 install python_speech_features

  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


## Preparación del entorno para poder correr mis módulos importados.

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

import sys
sys.path.append('drive/My Drive/Tesis')
sys.path.append('drive/My Drive/Tesis/repo')

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 2344332101687675252, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 8266229863755046234
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6002542210046975738
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11276946637
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 5711462065763696766
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]

# Entrenamiento de LASNet

## Importación de librerías necesarias

In [0]:
import tensorflow as tf
from src.utils.Database import Database
from src.utils.LASLabel import LASLabel
from src.utils.ProjectData import ProjectData
import time
from src.neural_network.LAS.LASNetData import LASNetData
import pprint
from src.Estimators.las.model_fn import model_fn
from src.Estimators.las.data_input_fn import data_input_fn
import shutil

## Definición de los hiperparámetros de la red



In [4]:
# Load project data
project_data = ProjectData()

network_data = LASNetData()
network_data.model_path = 'drive/My Drive/Tesis/repo/' + project_data.LAS_NET_MODEL_PATH
network_data.checkpoint_path = 'drive/My Drive/Tesis/repo/' + project_data.LAS_NET_CHECKPOINT_PATH
network_data.tensorboard_path = 'drive/My Drive/Tesis/repo/' + project_data.LAS_NET_TENSORBOARD_PATH

network_data.num_classes = LASLabel.num_classes
network_data.num_features = 494
network_data.num_embeddings = 0
network_data.sos_id = LASLabel.SOS_INDEX
network_data.eos_id = LASLabel.EOS_INDEX

network_data.beam_width = 0

network_data.num_dense_layers_1 = 2
network_data.num_units_1 = [400] * network_data.num_dense_layers_1
network_data.dense_activations_1 = [tf.nn.relu] * network_data.num_dense_layers_1
network_data.batch_normalization_1 = True
network_data.keep_prob_1 = [0.7] * network_data.num_dense_layers_1
network_data.kernel_init_1 = [tf.truncated_normal_initializer(mean=0, stddev=0.1)] * network_data.num_dense_layers_1
network_data.bias_init_1 = [tf.zeros_initializer()] * network_data.num_dense_layers_1

network_data.listener_num_layers = 1
network_data.listener_num_units = [256] * network_data.listener_num_layers
network_data.listener_activation_list = [tf.nn.tanh] * network_data.listener_num_layers
network_data.listener_keep_prob_list = [0.8] * network_data.listener_num_layers

network_data.num_dense_layers_2 = 1
network_data.num_units_2 = [300]
network_data.dense_activations_2 = [tf.nn.relu] * network_data.num_dense_layers_2
network_data.batch_normalization_2 = True
network_data.keep_prob_2 = [0.7] * network_data.num_dense_layers_2
network_data.kernel_init_2 = [tf.truncated_normal_initializer(mean=0, stddev=0.1)] * network_data.num_dense_layers_2
network_data.bias_init_2 = [tf.zeros_initializer()] * network_data.num_dense_layers_2

network_data.attention_type = 'luong'       # 'luong', 'bahdanau'
network_data.attention_num_layers = 1
network_data.attention_size = None
network_data.attention_units = 256
network_data.attention_activation = tf.nn.tanh
network_data.attention_keep_prob = 0.8

network_data.kernel_regularizer = 0.0
network_data.sampling_probability = 0.2

network_data.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)

pprint.pprint(network_data.as_dict())

{'attention_activation': <function tanh at 0x7fbba0aa1840>,
 'attention_keep_prob': 0.8,
 'attention_num_layers': 1,
 'attention_size': None,
 'attention_type': 'luong',
 'attention_units': 256,
 'batch_normalization_1': True,
 'batch_normalization_2': True,
 'beam_width': 0,
 'bias_init_1': [<tensorflow.python.ops.init_ops.Zeros object at 0x7fbb93b6b898>,
 'bias_init_2': [<tensorflow.python.ops.init_ops.Zeros object at 0x7fbb93b6b860>],
 'checkpoint_path': 'drive/My '
                    'Drive/Tesis/repo/out/las_net/checkpoint/model.ckpt',
 'dense_activations_1': [<function relu at 0x7fbba08ca840>,
                         <function relu at 0x7fbba08ca840>],
 'dense_activations_2': [<function relu at 0x7fbba08ca840>],
 'eos_id': 2,
 'keep_prob_1': [0.7, 0.7],
 'keep_prob_2': [0.7],
 'kernel_init_1': [<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fbb93b6b8d0>,
 'kernel_init_2': [<tensorflow.python.ops.init_ops.TruncatedNormal object at 0x7fbb93b6b940>],
 'kernel_regular

# Configuración del dataset

In [0]:
from src.Estimators.las.data_input_fn import data_input_fn


# def data_input_fn(filenames, batch_size, parse_fn, shuffle_buffer, num_features, sos_id, eos_id, num_epochs=1):
# #     return data_input_fn(filenames, batch_size, parse_fn, shuffle_buffer, num_features, sos_id, eos_id, num_epochs)
#     dataset = tf.data.TFRecordDataset(filenames)
#     dataset = dataset.map(parse_fn)

#     dataset = dataset.map(
#         lambda feature, target, feat_len, target_len: (feature,
#                                                        tf.concat(([sos_id], target), 0),
#                                                        tf.concat((target, [eos_id]), 0),
#                                                        feat_len,
#                                                        target_len)
#     )

#     dataset = dataset.map(
#         lambda feature, target_in, target_out, feat_len, target_len: (feature,
#                                                                       target_in,
#                                                                       target_out,
#                                                                       feat_len,
#                                                                       tf.size(target_in, out_type=tf.int64)))

#     # dataset = dataset.map(
#     #     lambda feature, target_in, target_out, feat_len, target_len: (feature,
#     #                                                                   tf.cast(target_in, tf.int32),
#     #                                                                   tf.cast(target_out, tf.int32),
#     #                                                                   tf.cast(feat_len, dtype=tf.int32),
#     #                                                                   tf.cast(target_len, dtype=tf.int32)
#     #                                                                   )
#     # )

#     dataset = dataset.padded_batch(
#         batch_size=batch_size,
#         padded_shapes=((None, num_features), [None], [None], (), ()),
#         padding_values=(tf.constant(value=0, dtype=tf.float32),
#                         tf.constant(value=eos_id, dtype=tf.int64),
#                         tf.constant(value=eos_id, dtype=tf.int64),
#                         tf.constant(value=0, dtype=tf.int64),
#                         tf.constant(value=0, dtype=tf.int64),
#                         )
#     )
#     dataset = dataset.shuffle(shuffle_buffer).repeat(num_epochs)

#     # return dataset
#     iterator = dataset.make_one_shot_iterator()
#     next_element = iterator.get_next()

#     feature, target_in, target_out, feat_len, target_len = next_element

#     feat_len = tf.cast(feat_len, dtype=tf.int32)
#     target_in = tf.cast(target_in, tf.int32)
#     target_out = tf.cast(target_out, tf.int32)
#     target_len = tf.cast(target_len, tf.int32)

#     return {'feature': feature, 'feat_len': feat_len}, \
#            {'targets_outputs': target_out, 'targets_inputs': target_in, 'target_len': target_len}


# Configuración del modelo

In [0]:
import tensorflow as tf
from src.neural_network.network_utils import bidirectional_pyramidal_rnn, attention_layer, attention_decoder, \
    beam_search_decoder, greedy_decoder, edit_distance, attention_loss, dense_multilayer
from src.Estimators.las.model_fn import model_fn


# def model_fn(features,
#              labels,
#              mode,
#              params):
# #   return model_fn(features, labels, mode, params)

#     input_features = features['feature']
#     input_features_length = features['feat_len']

#     decoder_inputs = None
#     targets = None
#     targets_length = None

#     if mode != tf.estimator.ModeKeys.PREDICT:
#         decoder_inputs = labels['targets_inputs']
#         targets = labels['targets_outputs']
#         targets_length = labels['target_len']

#     with tf.name_scope("dense_layer_1"):
#         input_features = dense_multilayer(input_ph=input_features,
#                                           num_layers=params['num_dense_layers_1'],
#                                           num_units=params['num_units_1'],
#                                           name='dense_layer_1',
#                                           activation_list=params['dense_activations_1'],
#                                           use_batch_normalization=params['batch_normalization_1'],
#                                           train_ph=mode == tf.estimator.ModeKeys.TRAIN,
#                                           use_tensorboard=True,
#                                           keep_prob_list=params['keep_prob_1'],
#                                           kernel_initializers=params['kernel_init_1'],
#                                           bias_initializers=params['bias_init_1'],
#                                           tensorboard_scope='dense_layer_1')

#     with tf.variable_scope('listener'):
#         listener_output, input_features_length, listener_state = bidirectional_pyramidal_rnn(
#             input_ph=input_features,
#             seq_len_ph=input_features_length,
#             num_layers=params['listener_num_layers'],
#             num_units=params['listener_num_units'],
#             name="listener",
#             activation_list=params['listener_activation_list'],
#             use_tensorboard=True,
#             tensorboard_scope="listener",
#             keep_prob=params['listener_keep_prob_list'],
#             train_ph=mode == tf.estimator.ModeKeys.TRAIN)
    
#     with tf.name_scope("dense_layer_2"):
#         listener_output = dense_multilayer(input_ph=listener_output,
#                                            num_layers=params['num_dense_layers_2'],
#                                            num_units=params['num_units_2'],
#                                            name='dense_layer_2',
#                                            activation_list=params['dense_activations_2'],
#                                            use_batch_normalization=params['batch_normalization_2'],
#                                            train_ph=mode == tf.estimator.ModeKeys.TRAIN,
#                                            use_tensorboard=True,
#                                            keep_prob_list=params['keep_prob_2'],
#                                            kernel_initializers=params['kernel_init_2'],
#                                            bias_initializers=params['bias_init_2'],
#                                            tensorboard_scope='dense_layer_2')

#     with tf.variable_scope('tile_batch'):
#         batch_size = tf.shape(listener_output)[0]
#         if mode == tf.estimator.ModeKeys.PREDICT and params['beam_width'] > 0:
#             listener_output = tf.contrib.seq2seq.tile_batch(
#                 listener_output, multiplier=params['beam_width'])
#             input_features_length = tf.contrib.seq2seq.tile_batch(
#                 input_features_length, multiplier=params['beam_width'])
#             listener_state = tf.contrib.seq2seq.tile_batch(
#                 listener_state, multiplier=params['beam_width'])
#             batch_size = batch_size * params['beam_width']

#     with tf.variable_scope('attention'):
#         attention_cell, attention_state = attention_layer(
#             input=listener_output,
#             lengths=input_features_length,
#             num_layers=params['attention_num_layers'],
#             attention_units=params['attention_units'],
#             attention_size=params['attention_size'],
#             attention_type=params['attention_type'],
#             activation=params['attention_activation'],
#             keep_prob=params['attention_keep_prob'],
#             train_ph=mode == tf.estimator.ModeKeys.TRAIN,
#             batch_size=batch_size,
#             input_state=None,
#             use_tensorboard=True,
#             tensorboard_scope='attention_cell'
#         )

#     with tf.variable_scope('speller'):
#         def embedding_fn(ids):
#             if params['num_embeddings'] != 0:
#                 target_embedding = tf.get_variable(
#                     name='target_embedding',
#                     shape=[params['num_classes'], params['num_embeddings']],
#                     dtype=tf.float32,
#                     initializer=tf.contrib.layers.xavier_initializer())
#                 return tf.nn.embedding_lookup(target_embedding, ids)
#             else:
#                 return tf.one_hot(ids, params['num_classes'])

#         projection_layer = tf.layers.Dense(params['num_classes'], use_bias=True, name='projection_layer')

#         maximum_iterations = None
#         if mode != tf.estimator.ModeKeys.TRAIN:
#             max_source_length = tf.reduce_max(input_features_length)
#             maximum_iterations = tf.to_int32(tf.round(tf.to_float(max_source_length) * 2))

#         if mode == tf.estimator.ModeKeys.TRAIN:
#             decoder_inputs = embedding_fn(decoder_inputs)

#             decoder = attention_decoder(
#                 input_cell=attention_cell,
#                 initial_state=attention_state,
#                 embedding_fn=embedding_fn,
#                 seq_embedding=decoder_inputs,
#                 seq_embedding_len=targets_length,
#                 projection_layer=projection_layer,
#                 sampling_prob=params['sampling_probability'])

#         elif mode == tf.estimator.ModeKeys.PREDICT and params['beam_width'] > 0:
#             decoder = beam_search_decoder(
#                 input_cell=attention_cell,
#                 embedding=embedding_fn,
#                 start_token=params['sos_id'],
#                 end_token=params['eos_id'],
#                 initial_state=attention_state,
#                 beam_width=params['beam_width'],
#                 projection_layer=projection_layer,
#                 batch_size=batch_size)
#         else:

#             decoder = greedy_decoder(
#                 inputs=attention_cell,
#                 embedding=embedding_fn,
#                 start_token=params['sos_id'],
#                 end_token=params['eos_id'],
#                 initial_state=attention_state,
#                 projection_layer=projection_layer,
#                 batch_size=batch_size)

#         decoder_outputs, final_context_state, final_sequence_length = tf.contrib.seq2seq.dynamic_decode(
#             decoder, maximum_iterations=maximum_iterations)

#     with tf.name_scope('prediction'):
#         if mode == tf.estimator.ModeKeys.PREDICT and params['beam_width'] > 0:
#             logits = tf.no_op()
#             sample_ids = decoder_outputs.predicted_ids
#         else:
#             logits = decoder_outputs.rnn_output
#             sample_ids = tf.to_int32(tf.argmax(logits, -1))

#     if mode == tf.estimator.ModeKeys.PREDICT:
#         predictions = {'sample_ids': sample_ids}

#         return tf.estimator.EstimatorSpec(mode, predictions=predictions)

#     with tf.name_scope('metrics'):
#         ler = edit_distance(
#             sample_ids, targets, params['eos_id'], None) #params.mapping)

#         metrics = {'LER': tf.metrics.mean(ler),}

#     tf.summary.scalar('LER', metrics['LER'][1])

#     with tf.name_scope('loss'):
#         kernel_loss = 0
#         for var in tf.trainable_variables():
#             if var.name.startswith('dense_layer') and 'kernel' in var.name:
#                 kernel_loss += tf.nn.l2_loss(var)

#         attetion_loss = attention_loss(
#             logits=logits,
#             targets=targets,
#             logits_length=final_sequence_length,
#             targets_length=targets_length,
#             eos_id=params['eos_id'],
#             train_ph=mode == tf.estimator.ModeKeys.TRAIN)

#         loss = attetion_loss + params['kernel_regularizer'] * kernel_loss

#     if mode == tf.estimator.ModeKeys.EVAL:
#         # with tf.name_scope('alignment'):
#         #     attention_images = utils.create_attention_images(
#         #         final_context_state)

#         # attention_summary = tf.summary.image(
#         #     'attention_images', attention_images)

#         # eval_summary_hook = tf.train.SummarySaverHook(
#         #     save_steps=10,
#         #     output_dir=os.path.join(config.model_dir, 'eval'),
#         #     summary_op=attention_summary)

#         logging_hook = tf.train.LoggingTensorHook(
#             tensors={
#                 'LER': tf.reduce_mean(ler),
#                 #'max_predictions': sample_ids[tf.argmax(ler)],
#                 #'max_targets': targets[tf.argmax(ler)],
#             },
#             every_n_iter=10)

#         return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics, evaluation_hooks=[logging_hook])

#     with tf.name_scope('train'):
#         optimizer = params['optimizer']
#         # optimizer = tf.train.AdamOptimizer(params.learning_rate)
#         train_op = optimizer.minimize(
#             loss, global_step=tf.train.get_global_step())

#     logging_hook = tf.train.LoggingTensorHook(
#         tensors={
#             'loss': loss,
#             'LER': tf.reduce_mean(ler)},
#         every_n_secs=10)

#     return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op, training_hooks=[logging_hook])


## Configuración de entrenamiento

In [8]:
model_dir = 'drive/My Drive/Tesis/repo/out/las_net/estimator/'

base_path = 'drive/My Drive/Tesis/repo/data/tfrecords/librispeech/las/ds_dataset/'

index_files = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]#, 15, 16, 17, 18, 19, 20, 21]
train_files = ['train_database_{}.tfrecords'.format(item) for item in index_files]
val_files = ['test_database_1.tfrecords', 'test_database_2.tfrecords']
test_files = ['test_database_2.tfrecords']

train_files = list(map(lambda x: base_path + x, train_files))
val_files = list(map(lambda x: base_path + x, val_files))
test_files = list(map(lambda x: base_path + x, test_files))

train_batch_size = 100
train_epochs = 1000

validate_batch_size = 100

restore_run = True

config = tf.estimator.RunConfig(
    model_dir=model_dir,
    save_checkpoints_steps=100,
    save_summary_steps=100,
    log_step_count_steps=100)


model = tf.estimator.Estimator(
    model_fn=model_fn,
    params=network_data.as_dict(),
    config=config
)

tf.logging.set_verbosity(tf.logging.INFO)

if not restore_run:
    shutil.rmtree(model_dir)


INFO:tensorflow:Using config: {'_model_dir': 'drive/My Drive/Tesis/repo/out/las_net/estimator/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fbb93b6b240>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


## Entrenamiento

### Entrenamiento

In [0]:
model.train(
    input_fn=lambda: data_input_fn(
        filenames=train_files,
        batch_size=train_batch_size,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=10,
        num_features=network_data.num_features,
        num_epochs=train_epochs,
        eos_id=LASLabel.EOS_INDEX,
        sos_id=LASLabel.SOS_INDEX
        )
)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from drive/My Drive/Tesis/repo/out/las_net/estimator/model.ckpt-286
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 286 into drive/My Drive/Tesis/repo/out/las_net/estimator/model.ckpt.
INFO:tensorflow:loss = 2.5610135, step = 286
INFO:tensorflow:LER = 0.8675544, loss = 2.5610135
INFO:tensorflow:LER = 0.87648565, loss = 2.5647058 (24.646 sec)
INFO:tensorflow:LER = 0.7996581, loss = 2.5664563 (22.840 sec)
INFO:tensorflow:LER = 0.8982411, loss = 2.5689673 (22.775 sec)
INFO:tensorflow:LER = 0.91725177, loss = 2.544212 (22.860 sec)
INFO:tensorflow:LER = 1.0375097, loss = 2.552


### Validación


In [0]:
model.evaluate(
    input_fn=lambda: data_input_fn(
        filenames=train_files,
        batch_size=validate_batch_size,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=1,
        num_features=network_data.num_features,
        eos_id=LASLabel.EOS_INDEX,
        sos_id=LASLabel.SOS_INDEX
    )
)

### Testeo

In [0]:
num_tests = 2

predictions = model.predict(
    input_fn=lambda: data_input_fn(
        filenames=test_files,
        batch_size=1,
        parse_fn=Database.tfrecord_parse_dense_fn,
        shuffle_buffer=1,
        num_features=network_data.num_features,
        eos_id=LASLabel.EOS_INDEX,
        sos_id=LASLabel.SOS_INDEX
    )
)
count = 0
for item in predictions:
  pred = item['sample_ids']
#   print(item)
  print("Predicted: " + LASLabel.from_index(pred))
  count += 1
  # print(count)
  if count >= num_tests:
    break